In [1]:
import pandas as pd
import numpy as np


# Get self assembly clustering results

In [2]:
runID = 'run1'
chromosomeID = 'chr17'
results_file = pd.read_csv(f"../{runID}-{chromosomeID}.txt")

FileNotFoundError: [Errno 2] File ../run1-chr17.txt does not exist: '../run1-chr17.txt'

# Get modified Nanopore reads
Note: update clustering script to save the dataframe of the nanopore reads. Also make sure the window # is saved in the .txt file. 

In [3]:
nanopore_reads = pd.read_csv(f"../{runID}-{chromosomeID}_nanopore_reads.csv") # load nanopore reads with padded sequences and other attribute data

<function pandas.io.parsers._make_parser_function.<locals>.parser_f(filepath_or_buffer: Union[str, pathlib.Path, IO[~AnyStr]], sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, cache_dates=True, iterator=False, chunksize=None, compression='infer', thousands=None, decimal: str = '.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)>

# Perform person assignment

In [ ]:
# Add column(s)
nanopore_reads['individual'] = np.nan
results_file['individual'] = np.nan

- NOTE: This next step assumes there are no gaps
- NOTE: We need to add a header to the text file so it has columns. We can also save it as a csv in the other file if that is easier e.g. convert to df then save as .csv

In [6]:
def propogate_assignment_1(read):
    ID = read['readID']
    individual = read['individual']
    
    nanopore_reads.loc[nanopore_reads.ID == ID, 'individual'] = individual # propogate the person assignment to the main reads dataframe
    results_file.loc[results_file.ID == ID, 'individual'] = individual # propogate the person assignment to the main reads dataframe
    ID_person_match[ID] = individual # add result to dictionary of results


In [7]:
def propogate_assignment_2(read_result):
    ID = read.readID
    individual = read.Individual
    
    if ID in ID_person_match.keys():
        # ID has already been assigned 
        # NOTE: same as "if np.isnan(read_result['individual']) == False"
        return None
    else:
        # Get cluster assignment of read
        cluster_num = temp.loc[temp.readID == ID, 'kmeans_cls2']
        
        # see the individual assigned to IDs in this cluster in this particular window
        cluster_assignment = temp.loc[temp.kmeans_cls2 == cluster_num]
        assigned_individual = cluster_assignment.individual.unique()
        
        if len(assigned_individual) > 1:
            # this cluster has mixed assignment which is bad
            # we expect that if IDs are already assigned they were in the same cluster in the previous window(s)
            # which means that this issue should theoretically never happen (NOTE: think about/check this logic)
            return None # UPDATE THIS
        
        elif len(assigned_individual) == 1:
            # we have our person
            individual_to_assign = assigned_individual.values[0] # check this syntax
            for ID in cluster_assignment['readID'].unique():
                if np.isnan(nanopore_reads.loc[nanopore_reads.ID == ID, 'individual']) == True:
                    nanopore_reads.loc[nanopore_reads.ID == ID, 'individual'] = individual_to_assign # propogate the person assignment to the main reads dataframe
                    results_file.loc[results_file.ID == ID, 'individual'] = individual_to_assign # propogate the person assignment to the main reads dataframe
                    ID_person_match[ID] = individual_to_assign # add result to dictionary of results


                            
        

In [ ]:
ID_person_match = {}
count = 0
for window in results_file['window_num'].unique: # iterate through instances of the sliding window 
    temp = results_file.loc[results_file['window_num']==window] # get data only from this window
    
    # We now have readIDs, cluster decisitions (0,1) for every read that was included in this window.
    # Step 1: We make a decision in the first window
    # Step 2: We perform ID bookeeping and cluster association to manage all following windows
    
    
    if count = 0:
        # Make decision in first window
        temp.loc[temp.kmeans_cls2 == 0, 'individual'] = 'person1' 
        temp.loc[temp.kmeans_cls2 == 1, 'individual'] = 'person2'
        
        temp.apply(lambda x: propogate_assignment_1(x), axis=1) # add results to main reads file
        
    if count != 0:
        # Book-keeping:
        # Step 1: make sure IDs match for ones that are already claimed
        # Step 2: if an ID has not been assigned determine its cluster (0,1) assign the associated individual based on already assigned reads in the cluster
        temp.apply(lambda x: propogate_assignment_2(x), axis=1) # add results to main reads file
        
        
    count += 1

# Save results

In [ ]:
ID_individual_mapping = pd.DataFrame(ID_person_match.items()) 
ID_individual_mapping.to_csv('./ID_individual_matches.csv')


In [ ]:
nanopore_reads.to_csv('./final_nanopore_reads.csv')